In [11]:
import os
import pandas as pd

In [12]:
BASE_DATA = 'data'
PROCESSED_DATA = 'processed_data'

MOTHES = ['Mar', 'May', 'Jul', 'Sep']

In [13]:
for filename in os.listdir(BASE_DATA):
    if not filename.startswith("Futures prices"):
        continue
    df = pd.read_excel(os.path.join(BASE_DATA, filename))
    df['Date'] = pd.to_datetime(df['Date'], format='%d %b %Y')
    columns = df.columns
    
    comodadity = filename.split()[4]
    if not os.path.exists(f"{PROCESSED_DATA}/{comodadity}"):
        os.makedirs(f"{PROCESSED_DATA}/{comodadity}")
        
    first_year = df['Date'].dt.year.min()
    last_year = df['Date'].dt.year.max()
    
    for year in range(first_year, last_year + 1):
        year_df = df[df['Date'].dt.year == year]
        rel_columns =  [col + ' ' + str(year + 1) for col in MOTHES]
        year_df = year_df[['Date'] + rel_columns]
        
        # column with the data diffrence between the current date and the previous date, the first has diff = 0
        year_df['diff'] = year_df['Date'].diff().dt.days.fillna(1).astype(int)
        
        year_df.to_csv(f"{PROCESSED_DATA}/{comodadity}/{comodadity}_{year}.csv", index=False)
        
        # check if there are missing data
        assert year_df.isna().sum().sum() == 0, f"Missing data in {comodadity}_{year}"
        

In [14]:
# create full dataset for each comodadity with prime
prime_df = pd.read_csv(os.path.join(PROCESSED_DATA, 'Prime.csv'))
prime_df
for filename in os.listdir(PROCESSED_DATA):
    if not os.path.isdir(os.path.join(PROCESSED_DATA, filename)):
        continue
    comodadity = filename
    df = pd.DataFrame()
    path = os.path.join(PROCESSED_DATA, comodadity)
    files = os.listdir(path)
    for file in files:
        year = file.split('_')[1].split('.')[0]
        temp_df = pd.read_csv(os.path.join(path, file))
        col_year = str(int(year) + 1)
        for col in temp_df.columns:
            if col_year in col:
                new_name = col.split(' ')[0]
                temp_df.rename(columns={col: new_name}, inplace=True)
        df = pd.concat([df, temp_df], axis=0)
    df = df.merge(prime_df, on='Date', how='left')
    df['Date'] = pd.to_datetime(df['Date'])
    df['diff'] = df['Date'].diff().dt.days.fillna(1).astype(int)
    df.to_csv(f"{PROCESSED_DATA}/full_{comodadity}.csv", index=False)

In [17]:
# after we created full dataset for each comodadity we can create a full dataset for all comodadities
full_df = pd.DataFrame()
for i, filename in enumerate(os.listdir(PROCESSED_DATA)):
    if not filename.startswith('full_'):
        continue
    temp_df = pd.read_csv(os.path.join(PROCESSED_DATA, filename))
    temp_df.drop(columns=['diff'], inplace=True)
    for col in temp_df.columns:
        if col in ['Date', 'Prime']:
            continue
        new_name = filename.split('_')[1].split('.')[0] + ' ' + col
        temp_df.rename(columns={col: new_name}, inplace=True)
    if i == 0:
        full_df = temp_df
    else:
        temp_df.drop(columns=['Prime'], inplace=True)
        full_df = pd.merge(full_df, temp_df, on='Date', how='inner')
full_df['Date'] = pd.to_datetime(full_df['Date'])
full_df['diff'] = full_df['Date'].diff().dt.days.fillna(1).astype(int)

prime_col = full_df['Prime']
full_df.drop(columns=['Prime'], inplace=True)
full_df['Prime'] = prime_col
full_df.to_csv(f"{PROCESSED_DATA}/full_dataset.csv", index=False)

In [19]:
# check no missing data
files = list(os.listdir(PROCESSED_DATA))
while files:
    file = files.pop()
    if os.path.isdir(os.path.join(PROCESSED_DATA, file)):
        add_files = os.listdir(os.path.join(PROCESSED_DATA, file))
        add_files = [os.path.join(file, add_file) for add_file in add_files]
        files += add_files
    else:
        # print(f"Checking {file}")
        df = pd.read_csv(os.path.join(PROCESSED_DATA, file))
        assert df.isna().sum().sum() == 0, f"Missing data in {file}"